# PROGRAMA PYTHON PARA RECÁLCULO DE PAGAMENTOS DE FÉRIAS DE SERVIDORES
## TRIBUNAL REGIONAL ELEITORAL DO PARÁ
## SEAUD - SECRETARIA DE AUDITORIA
## AUDITORIA DE CONTAS 2023 - CICLO DE DESPESAS COM PESSOAL
## TRILHA DE CONTROLE TR-DP.PAG-FER - Análise de Pagamento de Férias de Pessoal do TRE-PA

### Autores: Salomão Freitas Júnior (sfreitas@tre-pa.jus.br)
###          Marco Antônio Fagundes de Moraes (marco.fagundes@tre-pa.jus.br)
### 21/11/2023

### ESTE NOTEBOOK PYTHON CORRESPONDE À IMPLEMENTAÇÃO DO CASO 2- Exatidão dos valores de pagamentos de Férias - Recálculo, DO DOCUMENTO DE ESPECIFICAÇÃO DA TRILHA DE CONTROLE TR-DP.PAG-FER - Análise de Pagamento de Férias de Pessoal do TRE-PA

### ESTA VERSÃO TEM O O OBJETIVO DE DEMONSTRAÇÃO DA PRÁTICA DO PROJETO DE AUTOMATIZAÇÃO DE TESTES DE AUDITORIA BASEADA EM TRILHAS DE CONTROLE. O NOTEBOOK PYTHON UTILIZA COMO ENTRADA DADOS FICTÍCIOS, PRESERVANDO DADOS PESSOAIS DOS SERVIDORES DO TRE/PA.

## IMPORTAÇÃO DAS BIBLIOTECAS PRINCIPAIS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
import shutil

# DEFINIÇÃO DOS PATHS DE PASTAS E ARQUIVOS DE ENTRADA E SAÍDA

In [3]:
# ENTRADAS - FONTES DE DADOS
path_planilhao_rubricas = '/content/drive/MyDrive/Colab Notebooks/AUDITORIA DE CONTAS/2023 - Dados Fictícios/Análise Férias - Dados Fictícios/SPP - Planilhão Rubricas/Planilhão Rubricas 2023 - Dados Fictícios.xlsx'
planilha_resumo_analitico = r'/content/drive/MyDrive/Colab Notebooks/AUDITORIA DE CONTAS/2023 - Dados Fictícios/Análise Férias - Dados Fictícios/Resultados/Resumo_Analítico_Férias_Dados_Ficitícios.xlsx'

# SAÍDAS - RESULTADOS
path_resultados_ferias = r'/content/drive/MyDrive/Colab Notebooks/AUDITORIA DE CONTAS/2023 - Dados Fictícios/Análise Férias - Dados Fictícios/Resultados'
planilha_recalculos_ferias_ok = path_resultados_ferias + r'/Recálculos_Férias_Dados_Fictícios - OK.xlsx'
planilha_recalculos_ferias_nok = path_resultados_ferias + r'/Recálculos Férias_Dados_Fictícios - NOK.xlsx'


# LÊ AS PLANILHAS DE ENTRADA

In [4]:
# Lê o planilhão com as rubricas
rubricas_df = pd.read_excel(path_planilhao_rubricas)

display(rubricas_df)



,Mês/Ano Folha,N° Folha,Cod Rubrica,Descrição,Situação Servidor,Matrícula,Nome,CPF,Valor,Prazo Rubrica,Data Abertura
0,32023,7,59000,BANCO DE HORAS,ATIVO,MAT00001,Dra. Beatriz Macedo,000.000.000-01,1262.58,1,2023-03-06
1,122022,19,6015000,INSS,ATIVO,MAT00001,Dra. Beatriz Macedo,000.000.000-01,-29.24,1,2022-12-05
2,122022,19,59000,BANCO DE HORAS,ATIVO,MAT00001,Dra. Beatriz Macedo,000.000.000-01,389.90,1,2022-12-05
3,122022,17,6015000,INSS,ATIVO,MAT00001,Dra. Beatriz Macedo,000.000.000-01,-68.19,1,2022-12-05
4,122022,17,53000,HE-SECRETARIA-AGOSTO E NOVEMBRO,ATIVO,MAT00001,Dra. Beatriz Macedo,000.000.000-01,101.79,1,2022-12-05
...,...,...,...,...,...,...,...,...,...,...,...
112799,12023,1,224000,GAJ - INATIVOS,INATIVO,MAT01462,Heitor Martins,000.000.014-62,10909.22,999,2023-01-10
112800,12023,1,237000,ADICIONAL TEMPO DE SERVIÇO - INATIVOS,INATIVO,MAT01462,Heitor Martins,000.000.014-62,2259.76,999,2023-01-10
112801,12023,1,6104000,BANCO DO BRASIL - EMPRÉSTIMO,INATIVO,MAT01462,Heitor Martins,000.000.014-62,-9103.84,62,2023-01-10
112802,12023,1,6201000,PSSS INATIVO,INATIVO,MAT01462,Heitor Martins,000.000.014-62,-4138.85,1,2023-01-10


In [5]:
# Lê a planilha de resumo analítico, resultado gerado pelo notebook do caso 01
resumo_analitico_ferias_df = pd.read_excel(planilha_resumo_analitico)
display(resumo_analitico_ferias_df)

,MATRÍCULA,NOME_SERVIDOR,QTD_OCORRÊNCIA,ARQUIVO_01,PER_AQUISITIVO_01,VALOR_01,ARQUIVO_02,PER_AQUISITIVO_02,VALOR_02,ARQUIVO_03,PER_AQUISITIVO_03,VALOR_03,VALOR_DEVOLUCAO_01,FOLHA_DEVOLUCAO_01,VALOR_DEVOLUCAO_02,FOLHA_DEVOLUCAO_02,VALOR_DEVOLUCAO_03,FOLHA_DEVOLUCAO_03,VALOR_DEVOLUCAO_04,FOLHA_DEVOLUCAO_04
0,MAT01057,Heitor Dias,3,01 - Relação Por Banco FL 03 DEZ2022 (FÉRIA...,2023,7619.28,05 - Relação Por Banco FL 06 ABR2023 (FÉRIA...,NÃO ENCONTRADO,5869.66,06 - Relação Por Banco FL 14 MAI2023 (FÉRIA...,2023.0,5865.3,-6896.97,52023.0,-9310.24,42023.0,NaN,NaN,NaN,NaN
1,MAT00043,Sra. Bruna Moreira,3,01 - Relação Por Banco FL 03 DEZ2022 (FÉRIA...,2022,6310.28,09 - Relação Por Banco FL 08 AGO2023 (FÉRIA...,2023,6672.30,10 - Relação Por Banco FL 16 SET2023 (FÉRIA...,2023.0,6672.3,-7982.54,92023.0,-2501.58,42023.0,-2501.57,32023.0,-2501.57,22023.0
2,MAT01107,Alana Viana,2,04 - Relação Por Banco FL 12 MAR2023 (FÉRIA...,2022,8112.34,11 - Relação Por Banco FL 08 OUT2023 (FÉRIA...,2023,8147.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MAT00448,Alexia Souza,2,01 - Relação Por Banco FL 03 DEZ2022 (FÉRIA...,NÃO ENCONTRADO,339.72,05 - Relação Por Banco FL 06 ABR2023 (FÉRIA...,NÃO ENCONTRADO,360.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MAT00709,Antonella Fonseca,2,01 - Relação Por Banco FL 03 DEZ2022 (FÉRIA...,2022,4095.27,05 - Relação Por Banco FL 06 ABR2023 (FÉRIA...,NÃO ENCONTRADO,4299.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,MAT00489,Yasmin Cavalcanti,1,01 - Relação Por Banco FL 03 DEZ2022 (FÉRIA...,2023,6485.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
522,MAT01435,Yasmin Correia,1,03 - Relação Por Banco FL 04 FEV2023 (FÉRIA...,2023,8777.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
523,MAT00272,Zoe Fogaça,1,07 - Relação Por Banco FL 08 JUN2023 (FÉRIA...,2023,1985.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
524,MAT00427,Ágatha Viana,1,01 - Relação Por Banco FL 03 DEZ2022 (FÉRIA...,2023,4735.73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# FUNÇÕES ÚTEIS

In [6]:
# DEFINIÇÃO DE FUNÇÕES ÚTEIS

# função para obter o mês em formato numerico
def mes_numerico(mes_abrev):
  list_mes_abrev = ['JAN', 'FEV', 'MAR', 'ABR', 'MAI', 'JUN', 'JUL', 'AGO', 'SET', 'OUT', 'NOV', 'DEZ']

  return list_mes_abrev.index(mes_abrev) + 1


# FUNÇÃO PARA TRANSFORMAR TRUE/FALSE EM SIM/NÃO
def sim_nao(condicao):
  if condicao:
    return 'SIM'
  else:
    return 'NÃO'

# PROCESSAMENTO

###1. Para cada linha da planilha final (saída do caso 01), com os dados de matrícula do servidor, e número e mês da folha de cada ocorrência de pagamento, buscar na planilha de rubricas de folha de pagamento (planilhão), os pagamentos de ⅓ de férias e as rubricas que compõem a sua base de cálculo conforme os normativos vigentes.
###2. Recalcular os valores devidos de ⅓ de férias com base nas rubricas da base de cálculo obtidas e comparar com os valores efetivamente pagos.
###3. Consolidar os resultados em uma planilha, indicando para cada servidor se o pagamento efetivo foi compatível com o recálculo, a diferença entre os valores, e destacando eventuais divergências.

In [7]:
#PEGA AS VÁRIAS OCORRÊNCIAS DE FÉRIAS PARA UM MESMO SERVIDOR
# E CONSIDERA OS CASOS DE FOLHA DE FÉRIAS 122022, O MES DA FOLHA ORDINARIA 12023 (PARA BUSCAR OS VALORES DE BASE DE CALCULO),
# UMA VEZ QUE NÃO TEMOS OS DADOS DE FOLHA ORDINARIA DE 2022
# TAMBÉM ADICIONA À BUSCA NA FOLHA ORDINÁRIA AS RUBRICAS REFERENTES A PAGAMENTOS DE INATIVOS E DE SERVIDORES SEM VÍNCULO

## Gerar um dataframe de saída com o resumo do recálculo, contendo as colunas 'MATRÍCULA', 'NOME_SERVIDOR', 'MES_ANO_FOLHA',
#uma coluna para cada rubrica da base de cálculo, uma coluna para o valor efetivamente pago, uma coluna para o valor recalculado,
#uma coluna com a diferença entre os valores e uma coluna 'RECALCULO_OK', que poderá conter os valores SIM ou NÃO
# (ir compondo o dataframe a cada iteração nas planilhas)

# GERA O DATAFRAME DE SAÍDA, QUE CONTERÁ O RESULTADO DA ANÁLISE DO RECÁLCULO
# CONTÉM TODAS AS RUBRICAS QUE ENTRAM NO CALCULO DO 1/3 DE FÉRIAS
colunas_recalculo = ['MATRÍCULA', 'NOME_SERVIDOR', 'MES_ANO_FOLHA', 'VENCIMENTO', 'GAJ', 'FC', 'CJ', 'AD TEMPO SERVIÇO',
                     'AQ PÓS GRADUAÇÃO', 'AQ TREINAMENTO', 'AD QUINTOS', 'AD LOCALIDADE','AQ GRADUAÇÃO', 'GAS','PARCELA COMPENSATÓRIA']

out_recalculo_df = pd.DataFrame(columns=colunas_recalculo)

#Ler cada linha da planilha resumo, localizando a matrícula e o nome do servidor. Na linha localizada, obter para cada coluna ‘ARQUIVO_XX’,
# o nome do arquivo, e o número e o mês/ano da folha de pagamento (estes dados podem ser obtidos do próprio nome do arquivo)

# Itera no DF resumo_analitico_ferias_df e vai fazendo o recalculo para cada servidor
for linha in range(len(resumo_analitico_ferias_df)):
    # Obtém matricula e nome do servidor da iteração
    matricula = resumo_analitico_ferias_df.loc[linha,'MATRÍCULA']
    nome_servidor = resumo_analitico_ferias_df.loc[linha,'NOME_SERVIDOR']

    #print(str(matricula) + ' - ' + nome_servidor)

    n_ocorrencias = int(resumo_analitico_ferias_df.loc[linha,'QTD_OCORRÊNCIA'])

    # ITERA PELA QUANTIDADE DE OCORRÊNCIAS DE FÉRIAS DO SERVIDOR
    for i in range(n_ocorrencias):

      # obtém o mês/ano e numero da folha (obtidos do nome do arquivo da folha para banco -  coluna 'ARQUIVO_01',  'ARQUIVO_02', etc)
      nome_arquivo = str(resumo_analitico_ferias_df.loc[linha,'ARQUIVO_0' + str(i + 1)])
      n_folha = int(nome_arquivo[28:30])
      mes = nome_arquivo[31:34]
      mes_num = mes_numerico(mes)
      ano = nome_arquivo[34:38]
      mes_ano = int(str(mes_num) + str(ano))

      '''
      print(nome_arquivo)
      print(n_folha)
      print(mes)
      print(mes_num)
      print(ano)
      print(mes_ano)
      print()
      '''

      #Com os dados obtidos, filtrar no planilhão de rubricas as linhas correspondentes
      #ao conjunto (matrícula do servidor, número da folha, mês/ano da folha), correspondendo
      #com as colunas 'Matrícula', 'N° Folha' e , 'Mês/Ano Folha' do planilhão.
      #Este filtro corresponde às rubricas do servidor naquela folha

      #obtém um DF com os lançamentos no contracheque de férias do servidor
      contracheque_ferias_df = rubricas_df.loc[ (rubricas_df['Matrícula'] == matricula) & (rubricas_df['Mês/Ano Folha'] == mes_ano) & (rubricas_df['N° Folha'] == n_folha) ]

      #display(contracheque_ferias_df)

      #Na linha do planilhão de rubricas filtrado, obter o valor da rubrica correspondente
      #ao pagamento de ⅓ de férias (coluna “Descrição” = '1/3 FÉRIAS')

      #Obtenção do valor bruto pago de 1/3 de férias para este sewrvidor
      valor_1_3_ferias = float( contracheque_ferias_df.loc[contracheque_ferias_df['Descrição'] ==  (r'1/3 FÉRIAS'),  'Valor'] )

      #print(valor_1_3_ferias)


      # Em seguida, obter no planilhão de rubricas original, para aquele mesmo servidor e para aquele mesmo mês ano/folha,
      # e para folha de número 1 (folha ordinária) os valores de todas as rubricas que devem entrar na base de cálculo de férias
      # de acordo com os normativos vigentes, localizando pela coluna 'Cod Rubrica'
      # (in [1000, 2000, 6000, 18000, 20000, 22000, 23000, 25000, 84000, 238000, 239000,
      #60000, 5000, 201000, 248000, 109000, 237000, 224000, 218000, 210000, 230000, 24000, 7000] )

      #Obtenção dos valores base do servidor da iteração para o recálculo em um DF
      # BUSCA TODAS AS RUBRICAS QUE ENTRAM NO CALCULO DO 1/3 DE FÉRIAS (BUSCA NA FOLHA 1 - QUE É A ORDINÁRIA).
      # PEGA SÓ AS RUBRICAS COM PRAZO 999. POIS AS VEZES VEM ESSAS MESMAS RUBRICAS COM PRAZO =1, QUE SIGNIFICA NORMALMENTE VALORES DE PAGAMENTO DE DIFERENÇAS PROPORCIONAIS (RESTOS DE MESES ANTERIORES AJUSTADOS)
      # AS RUBRICAS 999 SÃO PERMANENTES, ESSAS QUE ENTRAM EM BASE DE CÁLCULO PARA 1/3 DE FÉRIAS

      #Caso o mes_ano das férias seja 122022, então vai buscar essas informnações na folha ordinária de janeiro/2023 (PORQUE NÃO TENHO OS DADOS DESSA FOLHA DE DEZ/2022)
      #senão ele busca na mesma folha das férias
      mes_ano_ordinaria = mes_ano
      if mes_ano == 122022:
        mes_ano_ordinaria = 12023

      valores_base_df = rubricas_df.loc[ (rubricas_df['Matrícula'] == matricula) & (rubricas_df['Mês/Ano Folha'] == mes_ano_ordinaria)  & (rubricas_df['N° Folha'] == 1) & (rubricas_df['Prazo Rubrica'] == 999) &
                                         (
                                          (rubricas_df['Cod Rubrica'] == 1000) | (rubricas_df['Cod Rubrica'] == 2000) | (rubricas_df['Cod Rubrica'] == 6000) |(rubricas_df['Cod Rubrica'] == 18000) |
                                          (rubricas_df['Cod Rubrica'] == 20000) | (rubricas_df['Cod Rubrica'] == 22000) | (rubricas_df['Cod Rubrica'] == 23000) | (rubricas_df['Cod Rubrica'] == 25000) |
                                          (rubricas_df['Cod Rubrica'] == 84000) | (rubricas_df['Cod Rubrica'] == 238000) | (rubricas_df['Cod Rubrica'] == 239000) | (rubricas_df['Cod Rubrica'] == 60000) |
                                          (rubricas_df['Cod Rubrica'] == 5000) | (rubricas_df['Cod Rubrica'] == 201000) | (rubricas_df['Cod Rubrica'] == 248000) | (rubricas_df['Cod Rubrica'] == 109000) |
                                          (rubricas_df['Cod Rubrica'] == 237000) | (rubricas_df['Cod Rubrica'] == 224000) | (rubricas_df['Cod Rubrica'] == 218000) | (rubricas_df['Cod Rubrica'] == 210000) |
                                          (rubricas_df['Cod Rubrica'] == 230000) | (rubricas_df['Cod Rubrica'] == 24000) | (rubricas_df['Cod Rubrica'] == 7000)
                                        )
                                      ]


      #display(valores_base_df)


      # OBTEM OS VALORES DOS LANÇAMENTOS BASE DO CÁLCULO - TRATANDO O CASO DE NÃO POSSUIR AQUELA RUBRICA (SETANDO COMO 0-ZERO)

      # VENCIMENTO (1000	VENCIMENTO	ATIVO) ou  (201000	PROVENTOS	INATIVO)
      if len(valores_base_df.loc[ (valores_base_df['Cod Rubrica'] == 1000) | (valores_base_df['Cod Rubrica'] == 201000) ] ) != 0:
        #vencimento = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 1000, 'Valor'] )
        vencimento = float(valores_base_df.loc[(valores_base_df['Cod Rubrica'] == 1000) | (valores_base_df['Cod Rubrica'] == 201000), 'Valor'].sum() )
      else:
        vencimento = 0.0

      #GAJ  (2000	GAJ	ATIVO) ou  (224000 GAJ-INATIVOS	INATIVO	)
      if len(valores_base_df.loc[(valores_base_df['Cod Rubrica'] == 2000) | (valores_base_df['Cod Rubrica'] == 224000)] ) != 0:
        #gaj = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 2000, 'Valor'] )
        gaj = float(valores_base_df.loc[(valores_base_df['Cod Rubrica'] == 2000) | (valores_base_df['Cod Rubrica'] == 224000), 'Valor'].sum() )
      else:
        gaj = 0.0

      #FC (6000	FUNÇÃO COMISSIONADA	ATIVO	)
      if len(valores_base_df.loc[ valores_base_df['Cod Rubrica'] == 6000 ] ) != 0:
        #fc = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 6000, 'Valor'] )
        fc = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 6000, 'Valor'].sum() )
      else:
        fc = 0.0

      #CJ (18000 CJ - servidor ATIVO ) ou (5000	CARGO EM COMISSÃO - INTEGRAL	 para servidores sem vínculo ocupante de CJ)
      if len(valores_base_df.loc[(valores_base_df['Cod Rubrica'] == 18000) | (valores_base_df['Cod Rubrica'] == 5000) ] ) != 0:
        #cj = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 18000, 'Valor'] )
        #cj = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 18000, 'Valor'].sum() )
        cj = float(valores_base_df.loc[(valores_base_df['Cod Rubrica'] == 18000) | (valores_base_df['Cod Rubrica'] == 5000), 'Valor'].sum() )
      else:
        cj = 0.0

      #AD TEMPO SERVIÇO (20000	ADICIONAL POR TEMPO DE SERVIÇO	ATIVO	) ou (237000	ADICIONAL TEMPO DE SERVIÇO - INATIVOS	INATIVO		)
      if len(valores_base_df.loc[ (valores_base_df['Cod Rubrica'] == 20000) |  (valores_base_df['Cod Rubrica'] == 237000) ] ) != 0:
        #ad_servico = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 20000, 'Valor'] )
        ad_servico = float(valores_base_df.loc[(valores_base_df['Cod Rubrica'] == 20000) |  (valores_base_df['Cod Rubrica'] == 237000), 'Valor'].sum() )
      else:
        ad_servico = 0.0

      #AQ PÓS GRADUAÇÃO(22000	ADICIONAL DE QUALIFICAÇÃO - PÓS	ATIVO	) ou (248000	ADICIONAL DE QUALIFICAÇÃO - PÓS - INATIVOS	INATIVO)
      if len(valores_base_df.loc[ (valores_base_df['Cod Rubrica'] == 22000) | (valores_base_df['Cod Rubrica'] == 248000) ] ) != 0:
        #aq_pos = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 22000, 'Valor'] )
        aq_pos = float(valores_base_df.loc[(valores_base_df['Cod Rubrica'] == 22000) | (valores_base_df['Cod Rubrica'] == 248000), 'Valor'].sum() )
      else:
        aq_pos = 0.0

      #AQ TREINAMENTO (23000	ADICIONAL DE QUALIFICAÇÃO - TREINAMENTO	ATIVO)
      if len(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 23000] ) != 0:
        #aq_treina = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 23000, 'Valor'] )
        aq_treina = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 23000, 'Valor'].sum() )
      else:
        aq_treina = 0.0

      #AD QUINTOS (25000	V.P.N.I. (QUINTOS/DÉCIMOS)	ATIVO	) ou (218000	V.P.N.I. (QUINTOS/DÉCIMOS)	INATIVO	)
      if len(valores_base_df.loc[ (valores_base_df['Cod Rubrica'] == 25000) | (valores_base_df['Cod Rubrica'] == 218000) ] ) != 0:
        #aq_quintos = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 25000, 'Valor'] )
        ad_quintos = float(valores_base_df.loc[(valores_base_df['Cod Rubrica'] == 25000) | (valores_base_df['Cod Rubrica'] == 218000), 'Valor'].sum() )
      else:
        ad_quintos = 0.0

      #AD LOCALIDADE (84000	GEL VPNI	ATIVO)
      if len(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 84000] ) != 0:
        #ad_local = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 84000, 'Valor'] )
        ad_local = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 84000, 'Valor'].sum() )
      else:
        ad_local = 0.0

      #AQ GRADUAÇÃO
      if len(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 238000] ) != 0:
        #aq_grad = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 238000, 'Valor'] )
        aq_grad = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 238000, 'Valor'].sum() )
      else:
        aq_grad = 0.0

      #GAS
      if len(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 239000] ) != 0:
        #gas = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 239000, 'Valor'] )
        gas = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 239000, 'Valor'].sum() )
      else:
        gas = 0.0

      #PARCELA COMPENSATÓRIA (109000	PARCELA COMPENSATÓRIA ATIVOS	ATIVO) OU (210000	PARCELA COMPENSATÓRIA	INATIVO	)
      if len(valores_base_df.loc[ (valores_base_df['Cod Rubrica'] == 109000) | (valores_base_df['Cod Rubrica'] == 210000) ] ) != 0:
        #gas = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 210000, 'Valor'] )
        p_compens = float(valores_base_df.loc[(valores_base_df['Cod Rubrica'] == 109000) | (valores_base_df['Cod Rubrica'] == 210000), 'Valor'].sum() )
      else:
        p_compens = 0.0

      #ADICIONAL DE INSALUBRIDADE  (60000 ADICIONAL DE INSALUBRIDADE	ATIVO)
      if len(valores_base_df.loc[ valores_base_df['Cod Rubrica'] == 60000  ] ) != 0:
        ad_isalub = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 60000, 'Valor'].sum() )
      else:
        ad_isalub = 0.0

      #VPNI-ART.29 LEI 11094/2005  (230000 VPNI-ART.29 LEI 11094/2005	ATIVO)
      if len(valores_base_df.loc[ valores_base_df['Cod Rubrica'] == 230000  ] ) != 0:
        vpni_art_29 = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 230000, 'Valor'].sum() )
      else:
        vpni_art_29 = 0.0

      #GRATIFICAÇÃO DE ATIV. DE SEG. COM PSSS  (24000	GRATIFICAÇÃO DE ATIV. DE SEG. COM PSSS	ATIVO)
      if len(valores_base_df.loc[ valores_base_df['Cod Rubrica'] == 24000  ] ) != 0:
        gas_psss = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 24000, 'Valor'].sum() )
      else:
        gas_psss = 0.0


      #FUNÇÃO COMISSIONADA COM PSSS	  (7000	FUNÇÃO COMISSIONADA COM PSSS	ATIVO)
      if len(valores_base_df.loc[ valores_base_df['Cod Rubrica'] == 7000  ] ) != 0:
        fc_psss = float(valores_base_df.loc[valores_base_df['Cod Rubrica'] == 7000, 'Valor'].sum() )
      else:
        fc_psss = 0.0


      lista_base_calculo = [vencimento, gaj, fc, cj, ad_servico, aq_pos, aq_treina, ad_quintos, ad_local, aq_grad, gas, p_compens, ad_isalub, vpni_art_29, gas_psss, fc_psss ]
      #print(lista_base_calculo)



      # Geração do Resultado da análise para o servidor da iteração (acrescenta um linha no DF)
      #print([vencimento, gaj, fc, cj, ad_servico, aq_pos, aq_treina, aq_quintos, ad_local, aq_grad, gas ])
      nova_linha = {
                'MATRÍCULA': [matricula],
                'NOME_SERVIDOR': [nome_servidor],
                'MES_ANO_FOLHA': [mes_ano],
                'VENCIMENTO': [ vencimento ],
                'GAJ': [ gaj ] ,
                'FC':[ fc ] ,
                'CJ':  [ cj ] ,
                'AD TEMPO SERVIÇO': [ ad_servico ] ,
                'AQ PÓS GRADUAÇÃO': [ aq_pos ] ,
                'AQ TREINAMENTO': [ aq_treina ] ,
                'AD QUINTOS': [ ad_quintos ] ,
                'AD LOCALIDADE': [ ad_local ] ,
                'AQ GRADUAÇÃO': [ aq_grad ] ,
                'GAS': [ gas ],
                'PARCELA COMPENSATÓRIA': [p_compens],
                'AD INSALUBRIDADE': [ad_isalub],
                'VPNI-ART.29': [vpni_art_29],
                'GAS COM PSSS': [gas_psss],
                'FC COM PSSS': [fc_psss],
                'VALOR_PAGO_1/3 FÉRIAS': valor_1_3_ferias,

                # Com todos os valores obtidos, efetuar o recálculo do valor de ⅓ de férias ( somando todos os valores e dividindo por 3)
                'TOTAL_RECALCULO_FERIAS_BRUTO': round(sum(lista_base_calculo)/3, 2),

                'DIFERENCA': (valor_1_3_ferias - round(sum(lista_base_calculo)/3, 2) ),
                'RECALCULO_OK': sim_nao(round(sum(lista_base_calculo)/3, 2) == valor_1_3_ferias)
            }

      #print(nova_linha)


      out_recalculo_df = pd.concat([out_recalculo_df, pd.DataFrame(nova_linha)]) #, ignore_index=True)

      #break

display(out_recalculo_df)
print('SOMA DAS DIFERENÇAS: ' +  str( round(out_recalculo_df['DIFERENCA'].sum(), 2) ) )
print('SOMA DOS VALORES ABSOLUTOS DAS DIFERENÇAS: ' +  str( round(out_recalculo_df['DIFERENCA'].abs().sum(), 2) ) )

/tmp/ipython-input-235794504.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  valor_1_3_ferias = float( contracheque_ferias_df.loc[contracheque_ferias_df['Descrição'] ==  (r'1/3 FÉRIAS'),  'Valor'] )
/tmp/ipython-input-235794504.py:255: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out_recalculo_df = pd.concat([out_recalculo_df, pd.DataFrame(nova_linha)]) #, ignore_index=True)
/tmp/ipython-input-235794504.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  valor_1_3_ferias = float( contracheque_ferias_df.loc[contracheque_ferias_df['Descrição

,MATRÍCULA,NOME_SERVIDOR,MES_ANO_FOLHA,VENCIMENTO,GAJ,FC,CJ,AD TEMPO SERVIÇO,AQ PÓS GRADUAÇÃO,AQ TREINAMENTO,...,GAS,PARCELA COMPENSATÓRIA,AD INSALUBRIDADE,VPNI-ART.29,GAS COM PSSS,FC COM PSSS,VALOR_PAGO_1/3 FÉRIAS,TOTAL_RECALCULO_FERIAS_BRUTO,DIFERENCA,RECALCULO_OK
0,MAT01057,Heitor Dias,122022,7792.30,10909.22,0.00,8411.01,0.00,584.42,233.77,...,0.0,0.00,0.0,0.00,0.0,0.0,9310.24,9310.24,0.0,SIM
0,MAT01057,Heitor Dias,42023,8259.84,11563.78,0.00,0.00,0.00,619.49,247.80,...,0.0,0.00,0.0,0.00,0.0,0.0,6896.97,6896.97,0.0,SIM
0,MAT01057,Heitor Dias,52023,8259.84,11563.78,0.00,0.00,0.00,619.49,165.20,...,0.0,0.00,0.0,0.00,0.0,0.0,6869.44,6869.44,0.0,SIM
0,MAT00043,Sra. Bruna Moreira,122022,7792.30,10909.22,3072.36,0.00,0.00,584.42,155.85,...,0.0,0.00,0.0,0.00,0.0,0.0,7504.72,7504.72,0.0,SIM
0,MAT00043,Sra. Bruna Moreira,82023,8259.84,11563.78,3256.70,0.00,0.00,619.49,247.80,...,0.0,0.00,0.0,0.00,0.0,0.0,7982.54,7982.54,0.0,SIM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,MAT00489,Yasmin Cavalcanti,122022,7792.30,10909.22,3072.36,0.00,0.00,584.42,233.77,...,0.0,0.00,0.0,0.00,0.0,0.0,7530.69,7530.69,0.0,SIM
0,MAT01435,Yasmin Correia,22023,8259.84,11563.78,0.00,8915.67,1156.38,619.49,247.80,...,0.0,61.16,0.0,0.00,0.0,0.0,10907.70,10907.70,0.0,SIM
0,MAT00272,Zoe Fogaça,62023,5034.29,0.00,0.00,0.00,100.68,377.57,100.69,...,0.0,0.00,0.0,0.00,0.0,0.0,1985.56,1985.56,0.0,SIM
0,MAT00427,Ágatha Viana,122022,4749.33,6649.06,3072.36,0.00,189.97,356.20,94.99,...,0.0,0.00,0.0,219.03,0.0,0.0,5124.68,5124.68,0.0,SIM


SOMA DAS DIFERENÇAS: 3266.06
SOMA DOS VALORES ABSOLUTOS DAS DIFERENÇAS: 5539.48


###4. Gerar como saída, duas planilhas, separando os casos em que não houve inconsistência daqueles em que foram identificadas diferenças.


In [9]:
# Gera os dataframes filtrados pela coluna ['RECALCULO_OK']

#recalculo OK
out_recalculo_OK_df = out_recalculo_df[out_recalculo_df['RECALCULO_OK'] == 'SIM']

#recalcculo NOK
out_recalculo_NOK_df = out_recalculo_df[out_recalculo_df['RECALCULO_OK'] == 'NÃO']

# Gera a planilha na Pasta de saída

In [10]:
#recalculo OK
out_recalculo_OK_df.to_excel(planilha_recalculos_ferias_ok, index = False, header = True)

#recalcculo NOK
out_recalculo_NOK_df.to_excel(planilha_recalculos_ferias_nok, index = False, header = True)